## Deploy model to an batch endpoint

In [ ]:
# Import credentials
from resource_credentials import (subscription_key, 
    resource_group_name, 
    workspace_name)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Create a ml client
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name,
                     workspace_name=workspace_name)

### Create endpoint

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

endpoint_name = "diabetes-model-batch-endpoint"

endpoint = BatchEndpoint(
    name=endpoint_name
)

ml_client.begin_create_or_update(endpoint).result()

### Configure and create deployment

In [ ]:
from azure.ai.ml.entities import Model, BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction

# Register local model
model = Model(
    path="./src/model",
    type=AssetTypes.MLFLOW_MODEL,
    name="mlflow-diabetes-model-local"
)

ml_client.models.create_or_update(model)

In [ ]:
# Create batch deployment
deployment = BatchDeployment(
    name="diabetes-model-batch-deployment",
    endpoint_name=endpoint_name,
    model=model,
    compute="compute-cluster-sdk",
    instance_count=1,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="BatchPredictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level= "info"
)

ml_client.batch_deployments.begin_create_or_update(deployment).result()

In [ ]:
# Set default deployment

endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

### Prepare data for batch predictions

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./src/batch-data"
dataset_name = "diabetes-data-for-batch"

dataset = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    name=dataset_name
)

ml_client.data.create_or_update(dataset)

dataset = ml_client.data.get(name=dataset_name, label="latest")

### Submit Batch Job

In [ ]:
from azure.ai.ml import Input

input = Input(type=AssetTypes.URI_FOLDER, path=dataset.id)

# Test endpoint
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=input
)

ml_client.jobs.get(job.name)

print(job)